In [31]:
from IPython.display import Javascript, display
import numpy as np
import pdb
from numpy import unravel_index
import json
from matplotlib import pyplot as plt
import scipy.io.wavfile as wavfile
from scipy.signal import savgol_filter
from scipy.signal import medfilt
from ipywidgets import widgets
import pandas as pd
from scipy.signal import resample

# Returns an int array with max values
def get_max_values (data, frequency_threshold = 0, sampling_rate = 0):    
    # Match the frequency threshold to an index in each column
    threshold_index = 0
    if frequency_threshold > 0 and sampling_rate > 0:
        threshold_index = int(frequency_threshold / ((sampling_rate / 2) / data.shape[0]))
    
    # Create a magnitude threshold which prevents low values from breaking up curves
    magnitude_threshold = 0.00005 * data.max()
    
    # Create an empty array of the same length as the data array
    max_array = np.zeros(data.shape[1], dtype=int)
    # Iterate through all columns of the FFT data
    previous_max_location = 0
    for i in range(0, data.shape[1]):
        col = data[threshold_index :, i]
        max_location = int(np.argmax(col) + threshold_index)
        if data[max_location, i] < magnitude_threshold:
            max_location = previous_max_location
        max_array[i] = max_location   #Location of the highest value, not the value itself    
        previous_max_location = max_location
    return max_array


# sampling_rate ÷ fft_n represents the number of rows in the data set.
# ... fft_n corresponds to the num of rows that would be in the data set if the symmetric half was not removed
# ... this is the same as calculating (sampling_rate / 2) ÷ (fft_n / 2)

# PRECISION NOTE: Both of these funcitons have a precision of around ~22Hz

def index_to_freq(index):
    return float(index) * ( float(sampling_rate) / float(fft_n) )

def freq_to_index(freq):
    return int( float(freq) / ( float(sampling_rate) / float(fft_n) ) )


def time_to_index(time, sampling_rate):    
    return int(float(time) * float(sampling_rate))

def index_to_time(index, sampling_rate):    
    return float(index) / float(sampling_rate)


def convert_file_to_file_path(atsfile):
    prefix = "Waveform/"

    if atsfile.find("ATS24.") >= 0:
        prefix += "ATS24/"
        tmp = atsfile.split(".")
        prefix += "24%02d.wf"%(int(tmp[1]))

    elif atsfile.find("ATS24*.") >= 0:
        prefix += "ATS24v2/"
        tmp = atsfile.split(".")
        prefix += "24%02d.wf"%(int(tmp[1]))

    elif atsfile.find("ATS26.") >= 0:
        prefix += "ATS26/"
        tmp = atsfile.split(".")
        prefix += "26%02d.wf"%(int(tmp[1]))

    elif atsfile.find("Prof") >= 0:
        prefix += "ISO23747/"
        prefix += atsfile + ".wf"

    elif atsfile.find("ISO2678") >= 0:
        prefix += "ISO26782/"
        tmp = atsfile.split(".")
        prefix += "26782%02d.wf"%(int(tmp[1]))

    elif atsfile.find("Custom") >= 0:
        tmp = atsfile.split(".")
        prefix += "Custom%02d.wf"%(int(tmp[1]))
    else:
        return None

    return prefix


def read_waveform_data(atsfile):

    if atsfile is None: # not a PWG waveform
        return None

    filepath = convert_file_to_file_path(atsfile)

    waveform_data = {}
    waveform_data["Header"]={}
    waveform_data["Parameters"]={}
    waveform_data["Data"]=[]
    currSection = ""
    with open(filepath) as f:
        data = f.readlines()
        for dline in data:
            sectionChanged = False
            # know the current section
            for sec in ["Header","Parameters","Data"]:
                if dline.find(sec) != -1:
                    currSection = sec
                    sectionChanged = True 
            if sectionChanged:
                continue 
                
            dline = dline.strip()
            if (currSection=="Header" or currSection=="Parameters") and len(dline)>1:
                keyval = dline.split("=")
                if keyval[1][0].isdigit():
                    keyval[1] = float(keyval[1])
                waveform_data[currSection][keyval[0]] = keyval[1]
            elif currSection=="Data" and len(dline)>1:
                waveform_data[currSection].append(float(dline))
    return waveform_data


# Global variables for use by dropdown call-back
global sampling_rate, raw_data, sampling_rate_columns, wav_form, meta, effort
# Original sampling rate divided by overlap since there will be a new window every [fft_shift] points 
# https://db.tt/4lOSyLeO 

# FFT Variables
# Number of points in each FFT window
fft_n = 2048

# Number of points each subsequent window is shifted
fft_shift = 128


In [32]:
# Open JSON file containing info about audio files

# data_path = 'audio_curve_data/Final_Data/'
# json_file = data_path + '010.json'

def load_json_data(data_path, json_file):
    global audio_files
    with open(json_file) as audio_data_json:    
        audio_files_data = json.load(audio_data_json)

    # Retrieve the name of all audio files for dropdown
    tests = audio_files_data['Tests']
    audio_files = {}
    audio_files_list = []
    for i in range(0, len(tests)):
        # IGNORE ALL YELLOW SIDESTACK TESTS
        if tests[i]['Sidestack'] != 'Sidestack C (Yellow)':
            test = tests[i]
            efforts = test['Efforts']
            for j in range(0, len(efforts)):
                effort = efforts[j]
                effort_data = {}
                file_name = effort['RecordedAudioFilenameForEffort']

                effort_data['WavFile'] = file_name

                ats_string = test['PWGFile']
                effort_data['PWGFile'] = ats_string

                if ats_string in audio_files:
                    next_val = 2
                    while ats_string + str(next_val) in audio_files:
                        next_val += 1
                    test_string = ats_string + '-' + str(next_val)
                else:
                    test_string = test['PWGFile']

                sidestack = test['Sidestack']
                downstream = test['DownstreamTube']
                ball = test['Ball']
                mouthpiece = test['Mouthpiece']
                effort_data['Metadata'] = sidestack + downstream + mouthpiece + ball
                effort_data['ID'] = test_string

                audio_files[test_string] = effort_data
                audio_files_list.append(test_string)


    # DEFAULT AUDIO FILE - If this file is changed, change the default file shown by the dropdown as well
    # Import data from an audio file, save sampling rate and raw data
    # default_file = '1464278801.365085.wav'

    audio_files_list.sort()

In [33]:
def run_algorithm():
    # Plot the raw spectrogram of the audio data 
    f = plt.figure(figsize=(15,10)) 
    plt.style.use('ggplot') 

    data, data_freq, data_time, _ = plt.specgram(raw_data, NFFT=fft_n, Fs=sampling_rate, noverlap=(fft_n-fft_shift) ) 
    # data is basically a matrix of magnitudes
    # data_freq is a single column with the same number of rows as data where each row is a frequency (0 - fs/2) 
    # data_time is an array of columns the same length as |ata where each index contains the time value at that column (0 - ~19) 
    data[data==0] = 0.0001 


    # Clear original figure 
    f.clear()

    # Plot a customized figure
    ax = plt.subplot(1,1,1) 
    ax.pcolorfast(data_time, data_freq, np.log(data+0.00001), cmap=plt.cm.bone) 
    plt.grid()
    plt.savefig('Results/Figures/' + effort['ID'] + ' - [1] Raw' + '.png')
    f.clear()
    
    frequency_threshold = 200
    max_values = get_max_values(data, frequency_threshold, sampling_rate)

    med_window_length = 31
    max_values = medfilt(max_values, med_window_length)

    sg_window_length = 21
    poly_order = 1
    max_values = savgol_filter(max_values, sg_window_length, poly_order, mode='mirror')

    max_values = max_values.round(decimals=0)
    max_values = max_values.astype(int)

    # Plot Modification
    f = plt.figure(figsize=(15,10))
    plt.style.use('ggplot')

    ax = plt.subplot(1,1,1)
    ax.pcolorfast(data_time, data_freq, np.log(data), cmap=plt.cm.bone)
    ax.plot(data_time, max_values*sampling_rate/fft_n)
    # max_values is a 1D array containing the vertical index which represents the max value for any given slice
    # ... because this value is NOT a frequency, we must map it to the correct frequency by multiplying the value by 
    # ... sampling_rate/fft_n which is generated by the following:
    # ... The FFT returns a symmetric graph from 0 to fft_n, but because it's symmetric, half must be removed (fft_n/2)
    # ... then to map a frequency to an index, the sampling rate must be divided by 2 (fs/2)... finally, to map the two,
    # ... any index from fft_n/2 must be mapped to fs/2, resuling in fs/2 ÷ fft_n/2 OR fs/fft_n

    # Calculate the overall energy by summing each column
    cols = 0
    db_data = np.log(data[(data_freq>200) & (data_freq<4000),:]) 
    energy = np.max(db_data,axis=cols)
    db_min = np.percentile(energy,10)
    energy = energy - db_min

    energy = np.mean(data, axis=cols)

    # Graph it on top of the spectrogram
    ax2 = ax.twinx()
    plt.ylim(0, max(energy))
    ax2.plot(data_time, energy, 'g')


    # Create variables for various features (curves)

    #REMOVE
    # Curve which will be used for the analysis
    # data_curve = max_values*sampling_rate/fft_n
    # data_curve = max_values
    # The data is multiplied by the sampling_rate/fft_n to map y-indices to frequencies

    # whistle_endpoints: the whistle at the beginning and end of the test
    whistle_endpoints = np.empty(data.shape[1])
    whistle_endpoints[:] = np.nan

    # whistle_midsection: the whistle (during vortex) after jumping in frequency
    whistle_midsection = np.empty(data.shape[1])
    whistle_midsection[:] = np.nan

    # whistle_combined_normalized: a curve combining all whistle components, including a normalized midsection
    whistle_combined_normalized = np.empty(data.shape[1])
    whistle_combined_normalized[:] = np.nan

    # vortex_fundamental: a curve representing the extracted vortex curve
    vortex_fundamental = np.empty(data.shape[1])
    vortex_fundamental[:] = np.nan

    # vortex_harmonics: an array for additional vortex harmonics (should they be detected)
    vortex_harmonics = []



    # Find point of max energy from the energy curve
    max_energy_loc = np.argmax(energy[0 : int(len(energy) * .8)])
    max_energy = energy[max_energy_loc]
    # max_energy_time = index_to_time(max_energy_loc, sampling_rate_columns)


    # Find # of index points in 1/2 second... create window of max_energy_location ± index
    search_window_time = 0.25
    search_window_shift = time_to_index(search_window_time, sampling_rate_columns)
    search_window_begin = max_energy_loc - search_window_shift
    search_window_end = max_energy_loc + search_window_shift + 1
    # +1 was added to account for python array indexing


    # Search through max_values to find the temp point of max magnitude (in the search window)
    init_max_mag_x = np.argmax(max_values[search_window_begin : search_window_end]) + search_window_begin
    init_max_mag_y = max_values[init_max_mag_x]
    midsection_lookahead = 0

    # Try to find top whistle if max_values didn't detect it

    if index_to_freq(init_max_mag_y) < 3000.0:
        midpoint = init_max_mag_x
        expansion = 10
        window_left = midpoint - expansion
        window_right = midpoint + expansion + 1
        window_bottom = freq_to_index(4500.0)
        window_top = freq_to_index(6000.0)

        search_area = data[window_bottom : window_top, window_left : window_right]

        top_whistle_peak = unravel_index(search_area.argmax(), search_area.shape)

        init_max_mag_y = top_whistle_peak[0] + window_bottom
        init_max_mag_x = top_whistle_peak[1] + window_left

        midsection_lookahead = 25
        print 'Performing alternate search for whistle midsection due to anomaly'

    # init_max_mag_time = index_to_time(init_max_mag_x, sampling_rate_columns)


    # Follow this curve within a band of ±30Hz and find end points to determine whistle mid-section
    search_index = init_max_mag_x - 1



    def identify_curve(data, curve, x_index, y_index, band_index, signal_threshold = 5.0, lookahead = 5):
        def get_signal_significance():
            if x_index >= data.shape[1] or x_index < 0:
                return 0.0, 0

            # INNER
            # Inner mean is the mean of the values surrounding the max within ± X Hz
            inner_lower_y = y_index - band_index
            if inner_lower_y < 0:
                inner_lower_y = 0

            inner_upper_y = y_index + band_index + 1
            if inner_upper_y >= data.shape[0]:
                inner_upper_y = data.shape[0] - 1

            inner_window = data[inner_lower_y : inner_upper_y, x_index]
            inner_mean = np.mean(inner_window)
    #         inner_mean = np.max(inner_window)

            # OUTER
            outer_band_index = band_index * 3
            outer_lower_y = inner_lower_y - outer_band_index
            if outer_lower_y < 0:
                outer_lower_y = 0

            outer_upper_y = inner_upper_y + outer_band_index
            if outer_upper_y > data.shape[0]:
                outer_upper_y = data.shape[0] - 1

            lower = data[outer_lower_y : inner_lower_y, x_index]
            upper = data[inner_upper_y : outer_upper_y, x_index]
            outer_window = np.concatenate([lower, upper], axis=0)

            outer_mean = np.mean(outer_window)
    #         outer_mean = np.max(outer_window)

            max_y_loc = np.argmax(inner_window) + inner_lower_y


            return inner_mean/outer_mean, max_y_loc

        curve_start = 0
        curve_end = len(data) - 1

        # Save original values
        orig_x = x_index
        orig_y = y_index
        orig_band_index = band_index

        # TRACE CURVE BACKWARDS
        signal_significance = 100.0
        end_found = False
        while not end_found:
    #         if x_index <= 0:
    #             break
            signal_significance, max_y_loc = get_signal_significance()
            if signal_significance >= signal_threshold:
                # Signal is still significant, continue following
                curve[x_index] = max_y_loc
                x_index -= 1
                y_index = max_y_loc
                # Resetting band in case it needed to be adjusted by the expanded search
                band_index = orig_band_index
            else:
                # Signal is no longer significant, look ahead and continue
                end_found = True
                curve_start = x_index + 1

                for i in range(0, lookahead):
                    x_index -= 1
    #                 if x_index > 0:
                    signal_significance, max_y_loc = get_signal_significance()
                    if signal_significance >= signal_threshold:
                        end_found = False
                        # Store the y-value so it can be found on the next full pass
                        y_index = max_y_loc

                            # Return back to index before so the next loop grabs the value
                        band_index = orig_band_index
                        break
    #                 else:
    #                     break

                if end_found:
                    x_index = curve_start
                    band_index *= 2
                    for i in range(0, lookahead):
                        x_index -= 1
                        signal_significance, max_y_loc = get_signal_significance()
                        if signal_significance >= signal_threshold:
                            end_found = False
                            # Store the y-value so it can be found on the next full pass
                            y_index = max_y_loc

                            # Return back to index before so the next loop grabs the value
                            break

                if end_found:
                    x_index = curve_start
                    band_index = orig_band_index
                    band_index /= 2
                    for i in range(0, lookahead):
                        x_index -= 1
                        signal_significance, max_y_loc = get_signal_significance()
                        if signal_significance >= signal_threshold:
                            end_found = False
                            # Store the y-value so it can be found on the next full pass
                            y_index = max_y_loc

                            # Return back to index before so the next loop grabs the value
                            break 

        # TRACE CURVE FORWARDS
        x_index = orig_x
        y_index = orig_y
        band_index = orig_band_index

        signal_significance = 100.0
        end_found = False
        while not end_found:
            signal_significance, max_y_loc = get_signal_significance()
            if signal_significance >= signal_threshold:
                # Signal is still significant, continue following
                curve[x_index] = max_y_loc
                x_index += 1
                y_index = max_y_loc
                # Resetting band in case it needed to be adjusted by the expanded search
                band_index = orig_band_index
            else:
                # Signal is no longer significant, look ahead and continue
                end_found = True
                curve_end = x_index - 1

                for i in range(0, lookahead):
                    x_index += 1
                    signal_significance, max_y_loc = get_signal_significance()
                    if signal_significance >= signal_threshold:
                        end_found = False
                        y_index = max_y_loc
                        break

                if end_found:
                    band_index *= 2
                    x_index = curve_end
                    for i in range(0, lookahead):
                        x_index += 1
                        signal_significance, max_y_loc = get_signal_significance()
                        if signal_significance >= signal_threshold:
                            end_found = False
                            y_index = max_y_loc
                            break

                if end_found:
                    band_index = orig_band_index
                    band_index /= 2
                    x_index = curve_end
                    for i in range(0, lookahead):
                        x_index += 1
                        signal_significance, max_y_loc = get_signal_significance()
                        if signal_significance >= signal_threshold:
                            end_found = False
                            y_index = max_y_loc
                            break


        return curve_start, curve_end


    # When investigating a dead signal, attempt to figure out where the next point SHOULD be...
    # - Slope of past 3 points
    # - Slope since origin
    # - Regression of curve so far


    x_index = init_max_mag_x
    y_index = init_max_mag_y

    band_index = freq_to_index(50.0)
    if midsection_lookahead > 0:
        band_index = freq_to_index(125.0)


    # WHISTLE MIDSECTION

    curve_start, curve_end = identify_curve(data, whistle_midsection, x_index, y_index, band_index, lookahead=midsection_lookahead)
    midsection_start = curve_start
    midsection_end = curve_end


    band_index = freq_to_index(50.0)


    # FRONT WHISTLE

    # Find a relevant y-index for for the beginning of the front whistle
    front_y_index = int(whistle_midsection[midsection_start] / 3)

    window_bottom = front_y_index - band_index
    window_top = front_y_index + band_index + 1
    window_left = midsection_start - 10
    window_right = midsection_start + 11

    front_search_area = data[window_bottom : window_top, window_left : window_right]
    front_peak = unravel_index(front_search_area.argmax(), front_search_area.shape)

    front_y_index = front_peak[0] + window_bottom
    front_x_index = front_peak[1] + window_left


    curve_start, curve_end = identify_curve(data, whistle_endpoints, front_x_index, front_y_index, band_index, lookahead=10)
    front_whistle_start = curve_start
    front_whistle_end = curve_end

    if np.isnan(front_whistle_start):
        front_whistle_start = midsection_start
        front_whistle_end = midsection_start




    # BACK WHISTLE

    # Find a relevant y-index for for the beginning of the back whistle
    end_y_index = int(whistle_midsection[midsection_end] / 3)

    window_bottom = end_y_index - band_index*2
    window_top = end_y_index + band_index*2 + 1
    window_left = midsection_end - 10
    window_right = midsection_end + 11

    back_search_area = data[window_bottom : window_top, window_left : window_right]
    back_peak = unravel_index(back_search_area.argmax(), back_search_area.shape)

    back_y_index = back_peak[0] + window_bottom
    back_x_index = back_peak[1] + window_left

    curve_start, curve_end = identify_curve(data, whistle_endpoints, back_x_index, back_y_index, band_index) #, lookahead=10)

    # if np.isnan(back_whistle_start):
    #     curve_start, curve_end = identify_curve(data, whistle_endpoints, midsection_end, front_y_index, band_index) #, lookahead=10)

    back_whistle_start = curve_start
    back_whistle_end = curve_end

    if np.isnan(back_whistle_start):
        back_whistle_start = midsection_end
        back_whistle_end = midsection_end



    # ADDITIONAL WHISTLE DATA POINTS
    whistle_min = np.nan
    if np.isnan(whistle_endpoints).all():
        whistle_min = int(np.nanmin(whistle_midsection))
    else:
        whistle_min = int(np.nanmin(whistle_endpoints))


    whistle_max = int(np.nanmax(whistle_midsection))


    midsection_min = int(np.nanmin(whistle_midsection))
    midsection_max = int(np.nanmax(whistle_midsection))


    def process_curve(curve, curve_start, curve_end, search_range_start, search_range_end, outer_range_start, outer_range_end):
        if np.isfinite(outer_range_start) and curve_start < outer_range_start:
            curve[ : outer_range_start] = np.nan
            curve_start = outer_range_start

        if np.isfinite(outer_range_end) and curve_end > outer_range_end:
            curve[outer_range_end + 1: ] = np.nan
            curve_end = outer_range_start

        curve[curve <= 10] = np.nan

        for i in range(curve_end, curve_start - 1, -1):
            if np.isfinite(curve[i]) == True:
                curve_end = i
                break

        curve_peak_x = np.nanargmax(curve[search_range_start : search_range_end]) + search_range_start
        curve_peak_y = np.nanmax(curve[search_range_start : search_range_end])


        window = 10

        start_front_idx = curve_peak_x - int((curve_peak_x - curve_start) * .3)
        start_back_idx = curve_peak_x + int((outer_range_end - curve_peak_x) * .3)

        # Cough detection
        back_idx_buffer = 5
        if start_back_idx + back_idx_buffer < curve_end:
            second_max = np.nanargmax(curve[start_back_idx + 5 : curve_end]) + start_back_idx + 5
            back_midpoint = np.min([start_back_idx + int((curve_end - start_back_idx) * .5), outer_range_end])

            if second_max < back_midpoint:
                start_back_idx = second_max + int((back_midpoint - second_max) * .3)

        for i in range(start_front_idx, curve_start, -1):
            if i - window < curve_start:
                slope = np.nanmean(np.diff(curve[curve_start : i + 1]))
                if slope < 0:
                    curve_start = i
                    break
            else:
                slope = np.nanmean(np.diff(curve[i - window : i + 1]))
                if slope < 0:
                    curve_start = i - window + 1
                    break


        for i in range(start_back_idx, curve_end + 1, 1):
            if i + window > curve_end:
                slope = np.nanmean(np.diff(curve[i : curve_end + 1]))
                if slope > 0:
                    curve_end = i
                    break
            else:
                slope = np.nanmean(np.diff(curve[i : i + window + 1]))

                if slope > 0:
                    curve_end = i + window - 1
                    break


        curve[ : curve_start] = np.nan
        curve[curve_end : ] = np.nan

        return curve, curve_start, curve_end


    # Define initial search area for vortex fundamental
    area_width = back_whistle_start - front_whistle_end
    midpoint = front_whistle_end + area_width/2
    expansion = int(area_width*.35)
    window_start = midpoint - expansion
    window_end = midpoint + expansion

    bottom_noise_index = 20
    search_area = data[bottom_noise_index : midsection_min - 1, window_start : window_end]
    # search_area = data[20 : midsection_min - 1, front_whistle_end : back_whistle_start]
    # search_area = data[20 : whistle_min - 1, front_whistle_end : back_whistle_start]

    # If front whistle continues across entire test
    if front_whistle_end > back_whistle_end or search_area.shape[1] <= 5:
        print "Search area modified due to whistle anomaly"
        search_area = data[bottom_noise_index : whistle_min - 1, midsection_start : midsection_end]
        window_start = midsection_start

    peak_vortex_mag = unravel_index(search_area.argmax(), search_area.shape)

    peak_vortex_mag_y = peak_vortex_mag[0] + bottom_noise_index
    peak_vortex_mag_x = peak_vortex_mag[1] + window_start

    # Modifying band_index significantly impacts the significance detection

    midsection_range = midsection_max - midsection_min

    core_data = np.copy(data)


    if front_whistle_start and front_whistle_end:
        core_data[whistle_min - 1 : , front_whistle_start - 1 : front_whistle_end + 1] = np.nan

    if back_whistle_start and back_whistle_end:
        core_data[whistle_min - 1 : , back_whistle_start - 1 : back_whistle_end + 1] = np.nan

    best_curve_weight = 0.0

    start_freq = 50.0
    stop_freq = 300.0
    step_freq = 25.0

    range_end = int((stop_freq - start_freq)/step_freq + 1)

    # Determine best vortex
    vortex_fundamentals = []
    for i in range(0, range_end, 1):
        curve = np.empty(data.shape[1])
        curve[:] = np.nan

        band_index = freq_to_index(start_freq + step_freq * i)

        curve_start, curve_end = identify_curve(core_data, curve, peak_vortex_mag_x, peak_vortex_mag_y, band_index, signal_threshold=1.2, lookahead = 10)
        raw_curve = np.copy(curve)



        if curve_start < curve_end:
            curve, curve_start, curve_end = process_curve(curve, curve_start, curve_end, midsection_start, midsection_end, front_whistle_start, back_whistle_end)
        curve_data = {'curve': curve, 'raw_curve': raw_curve, 'curve_start': curve_start, 'curve_end': curve_end}
        vortex_fundamentals.append(curve_data)

        num_finite = np.sum(np.isfinite(curve[curve_start : curve_end + 1]))
        curve_length = curve_end - curve_start
        if curve_length == 0:
            curve_density = 0
            curve_weight = 0
        else:
            curve_density = float(num_finite) / float(curve_length)
            curve_weight = float(curve_length) * float(curve_density)

        if  curve_weight > best_curve_weight:
            best_curve_weight = curve_weight
            best_curve = i

    print 'Best Curve Identified with freq %0.1f Hz and weight %.1f' % (start_freq + step_freq * best_curve, best_curve_weight * 100)
    best_curve_data = vortex_fundamentals[best_curve]
    vortex_fundamental = best_curve_data['curve']
    vortex_start = best_curve_data['curve_start']
    vortex_end = best_curve_data['curve_end']


    # NOTE: When trying to graph curves, either use the data_freq array as the y-axis labels
    # ... or multiply the data by sampling_rate/fft_n to map indices to frequencies
    data_curve = max_values * sampling_rate/fft_n

    fig = plt.figure(figsize=(15,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.set_xlabel('Index')
    ax.set_ylabel('Frequency')

    ax.plot(max_values, 'r')

    # ax.plot(search_window_begin, max_values[search_window_begin], 'og')
    # ax.text(search_window_begin, max_values[search_window_begin], 'Search Start')
    # ax.plot(search_window_end, max_values[search_window_end], 'or')
    # ax.text(search_window_end, max_values[search_window_end], 'Search Stop')

    # ax.plot(init_max_mag_x, init_max_mag_y, 'or')
    # ax.text(init_max_mag_x, init_max_mag_y - init_max_mag_y*0.01, 'Max Mag (Init)')


    # ax.plot(curve_end, init_y_index, 'or')
    # ax.text(curve_end, init_y_index, 'WAT')
    ax.plot(whistle_midsection, 'b')
    ax.plot(whistle_endpoints, 'b')

    # ax.plot(vortex_fundamental_copy, 'y')
    ax.plot(vortex_fundamental, 'b')
    # ax.plot(vortex_peak_x, vortex_peak_y, 'or', alpha = .3)
    # ax.plot(start_front_idx, vortex_fundamental[start_front_idx], 'og', alpha = .3)
    # ax.plot(start_back_idx, vortex_fundamental[start_back_idx], 'og', alpha = .2)
    # ax.plot(back_x_index, back_y_index, 'ob', alpha = .2)


    # Add time as a second x-axis
    ax2 = ax.twiny()
    ax2.set_xlim(0, data_time[-1])
    ax2.set_xlabel('Time')


    # Add energy as a second y-axis
    ax3 = ax.twinx()
    ax3.set_ylabel('Energy')

    ax3.plot(energy, 'g')
    ax3.plot(max_energy_loc, max_energy, 'ob')
    ax3.text(max_energy_loc, max_energy + 0.01*max_energy, 'Max Energy Value')


    f = plt.figure(figsize=(15,10))
    plt.style.use('ggplot')


    # data[20 : midsection_min - 1, midpoint - expansion : midpoint + expansion] *= 100

    ax = plt.subplot(2,1,1)
    # ax = plt.subplot(1,1,1)
    data[:bottom_noise_index, :] *= .1

    ax.pcolorfast(np.log(data), cmap=plt.cm.bone)
    ax2 = plt.subplot(2,1,2)
    ax2.pcolorfast(np.log(data), cmap=plt.cm.bone)


    ax.plot(whistle_midsection, 'b')
    ax.plot(whistle_endpoints, 'b')

    ax.plot(vortex_fundamentals[1]['curve'], 'r')
    ax.plot(vortex_fundamentals[1]['raw_curve'], 'y', linewidth = 2.0)



    # for curve in vortex_harmonics:
    #     ax.plot(curve, 'g')


    ax.plot(vortex_fundamental, 'g')


    plt.savefig('Results/Figures/' + effort['ID'] + ' - [3] Curve Following Comparison' + '.png')
    f.clear()

    f = plt.figure(figsize=(15,10))
    plt.style.use('ggplot')


    # back_search_area *= 10000

    # ax = plt.subplot(2,1,1)
    ax = plt.subplot(1,1,1)

    ax.pcolorfast(np.log(data), cmap=plt.cm.bone)

    ax.plot(whistle_midsection, 'b')
    ax.plot(whistle_endpoints, 'b')

    ax.plot(vortex_fundamental, 'r')

    plt.savefig('Results/Figures/' + effort['ID'] + ' - [2] Curve Following' + '.png')
    f.clear()




    # Create blended whistle signal
    whistle_combined_normalized = np.copy(whistle_endpoints)
    first_whistle_point = min(front_whistle_start, front_whistle_end, back_whistle_start, back_whistle_end)

    if first_whistle_point > midsection_start:
        diff = np.nanmin(whistle_midsection) - whistle_endpoints[back_whistle_start]
        whistle_combined_normalized[midsection_start : back_whistle_start + 1] = whistle_midsection[midsection_start : back_whistle_start + 1] - diff
    elif front_whistle_end < back_whistle_start:
        diff = np.nanmin(whistle_midsection) - np.nanmax([whistle_endpoints[front_whistle_end], whistle_endpoints[back_whistle_start]])
        whistle_combined_normalized[front_whistle_end : back_whistle_start + 1] = whistle_midsection[front_whistle_end : back_whistle_start + 1] - diff



    ax = plt.subplot(1,1,1)

    ax.pcolorfast(np.log(data), cmap=plt.cm.bone)
    ax.plot(whistle_combined_normalized)
    plt.savefig('Results/Figures/' + effort['ID'] + ' - [5] Blended Whistle' + '.png')
    f.clear()

    crop_start = 0

    temp = np.isfinite(whistle_midsection) | np.isfinite(whistle_endpoints) | np.isfinite(vortex_fundamental)

    for i in range(0, len(temp)):
        if temp[i] == 1:
            crop_start = max(i-300,0)

            break

    cropped_midsection = whistle_midsection[crop_start :]
    cropped_endpoints = whistle_endpoints[crop_start :]
    cropped_fundamental = vortex_fundamental[crop_start :]
    cropped_blended_whistle = whistle_combined_normalized[crop_start :]


    # plot the waveform of the test
    time = np.arange(0,len(wav_form["Data"])).astype(np.float)/float(wav_form["Header"]["Freq"])
    wav_fs = float(wav_form["Header"]["Freq"])


    # add proper axes
    if wav_form["Header"]["Type"] =="VT":

        volume = np.array(wav_form["Data"]).astype(np.float)
        flow = (volume[1:]-volume[:-1])*wav_fs
        flow = np.hstack(([0],flow))
        # apply filter to calced flow rate
        flow = savgol_filter(flow, 21, 2 )

    elif wav_form["Header"]["Type"] =="FT":
        flow = np.array(wav_form["Data"]).astype(np.float)
        volume = np.cumsum(flow/wav_fs)

    orig_flow = flow


    x1 = cropped_fundamental
    x2 = cropped_endpoints
    x3 = cropped_midsection
    x4 = cropped_blended_whistle

    # make same sampling rate
    flow = orig_flow
    flow = resample(flow,float(len(flow))/float(wav_fs)*float(sampling_rate_columns))

    x1_m = np.nanargmax(x1)
    f_m = np.nanargmax(flow)


    # align maxima
    if x1_m>f_m:
        flow = np.hstack((np.ones((x1_m-f_m,))*np.nan,flow))
    else:
        x1 = np.hstack((np.ones((f_m-x1_m,))*np.nan,x1))
        x2 = np.hstack((np.ones((f_m-x1_m,))*np.nan,x2))
        x3 = np.hstack((np.ones((f_m-x1_m,))*np.nan,x3))
        x4 = np.hstack((np.ones((f_m-x1_m,))*np.nan,x4))

    # make equal lengths
    if len(x1)>len(flow):
        flow = np.hstack((flow,np.ones((len(x1)-len(flow),))*np.nan))
    else:
        x1 = np.hstack((x1,np.ones((len(flow)-len(x1),))*np.nan))
        x2 = np.hstack((x2,np.ones((len(flow)-len(x2),))*np.nan))
        x3 = np.hstack((x3,np.ones((len(flow)-len(x3),))*np.nan))
        x4 = np.hstack((x4,np.ones((len(flow)-len(x4),))*np.nan))

    volume = np.cumsum(flow/sampling_rate_columns)
    df = pd.DataFrame(np.vstack((flow,volume,x1,x2,x3,x4)).T,columns=['flow','volume','vortex','side_f','side_h1','side_blended'])

    df['PWG'] = str(wav_form['Header']['Group'])+str(wav_form['Header']['Name'])
    df['meta'] = meta
    df.head()


    dftmp = df[['flow','vortex']]/df[['flow','vortex']].max()
    dftmp.plot()
    plt.savefig('Results/Figures/' + effort['ID'] + ' - [4] Flow vs Vortex' + '.png')
    f.clear()

    # effort = audio_files[json_selector.value]
    file_name = effort['WavFile']

    df.to_csv('Results/' + effort['ID'] + ' - ' + file_name + '.csv')
    print 'Results/' + effort['ID'] + ' - ' + file_name + '.csv'
    
    plt.close()

In [37]:

data_path = 'audio_curve_data/'
# data_path = 'audio_curve_data/Final_Data/'

# json_file = data_path + '105.json'
json_file = data_path + '107.json'
# json_file = data_path + '010.json'



# json_105 = ['ATS24.3','ATS24*.3','ATS24.4','ATS24*.4','ATS24.11','ATS24.13','ATS24*.13','ATS24.15','ATS24*.15','ATS24.17','ATS24.22','ATS24.23','ATS24.24','ATS26*.6','ATS26.7','ATS26*.7','ATS26.8','ATS26*.8','ATS26.25','ATS26*.25','ATS24.20','ATS24*.20','ATS26*.3','ATS26.21','ATS26*.21','ATS26.22','ATS26*.22','ATS24.8','ATS24*.8','ATS24.12','ATS26.2','ATS26*.2','ATS26.4','ATS26*.4','ATS26*.9','ATS26.10','ATS26*.10','ATS26.19','ATS26*.19','ATS26.23','ATS26*.23']

json_107 = ['ATS24.3','ATS24*.3','ATS24.4','ATS24*.4','ATS24.11','ATS24*.11','ATS24.13','ATS24.15','ATS24*.15','ATS24.17','ATS24*.17','ATS24.22','ATS24*.22','ATS24.23','ATS24*.23','ATS24.24','ATS26.6','ATS26*.6','ATS26.7','ATS26*.7','ATS26.8','ATS26*.8','ATS26.25','ATS26*.25','ATS24.20','ATS24*.20','ATS26.3','ATS26*.3','ATS26.21','ATS26*.21','ATS26.22','ATS26*.22','ATS24.8','ATS24*.8','ATS24.12','ATS26.2','ATS26*.2','ATS26.4','ATS26*.4','ATS26.9','ATS26*.9','ATS26.10','ATS26*.10','ATS26.19','ATS26*.19','ATS26.23','ATS26*.23','ATS26.26', 'ATS26*.26']

# json_010 = ['ATS24.20','ATS26.3','ATS26.21','ATS26.22','ATS24.1','ATS24.2','ATS24.5','ATS24.6','ATS24.7','ATS24.8','ATS24.9','ATS24.10','ATS24.12','ATS24.14','ATS24.16','ATS24.18','ATS24.19','ATS24.21','ATS26.1','ATS26.2','ATS26.4','ATS26.5','ATS26.9','ATS26.10','ATS26.11','ATS26.12','ATS26.13','ATS26.14','ATS26.15','ATS26.16','ATS26.17','ATS26.18','ATS26.19','ATS26.20','ATS26.23','ATS26.24']

# all_missing = ['ATS24.3','ATS24*.3','ATS24.4','ATS24*.4','ATS24.11','ATS24*.11','ATS24.13','ATS24*.13','ATS24.15','ATS24*.15','ATS24.17','ATS24*.17','ATS24.22','ATS24*.22','ATS24.23','ATS24*.23','ATS24.24','ATS24*.24','ATS26.6','ATS26*.6','ATS26.7','ATS26*.7','ATS26.8','ATS26*.8','ATS26.25','ATS26*.25','ATS24.20','ATS24*.20','ATS26.3','ATS26*.3','ATS26.21','ATS26*.21','ATS26.22','ATS26*.22','ATS24.8','ATS24*.8','ATS24.12','ATS24*.12','ATS26.2','ATS26*.2','ATS26.4','ATS26*.4','ATS26.9','ATS26*.9','ATS26.10','ATS26*.10','ATS26.19','ATS26*.19','ATS26.23','ATS26*.23','ATS26.26', 'ATS26*.26']


tests_to_run = json_107
# tests_to_run = ['ATS26.26', 'ATS26*.26']



load_json_data(data_path, json_file)



global sampling_rate, raw_data, sampling_rate_columns, wav_form, meta, effort, audio_files

# file = audio_files_list[0]

for test in tests_to_run:
    if test in audio_files:
        print test
        effort = audio_files[test]
        file_name = effort['WavFile']

        sampling_rate, raw_data = wavfile.read(data_path + file_name)
        sampling_rate_columns = sampling_rate / fft_shift
        wav_form = read_waveform_data(effort['PWGFile'])
        meta = effort['Metadata']

        run_algorithm()


print '\n\nFinished'


ATS24*.3
Performing alternate search for whistle midsection due to anomaly
Search area modified due to whistle anomaly
Best Curve Identified with freq 75.0 Hz and weight 46500.0
Results/ATS24*.3 - 1459459078.202079.wav.csv
ATS24*.4
Best Curve Identified with freq 50.0 Hz and weight 29300.0
Results/ATS24*.4 - 1459459128.589014.wav.csv
ATS24*.17
Best Curve Identified with freq 300.0 Hz and weight 31300.0
Results/ATS24*.17 - 1459459735.092742.wav.csv
ATS24*.22
Best Curve Identified with freq 125.0 Hz and weight 42500.0
Results/ATS24*.22 - 1459459910.151819.wav.csv
ATS24*.23
Search area modified due to whistle anomaly
Best Curve Identified with freq 50.0 Hz and weight 43500.0
Results/ATS24*.23 - 1459459966.974051.wav.csv
ATS26.6
Performing alternate search for whistle midsection due to anomaly
Search area modified due to whistle anomaly
Best Curve Identified with freq 75.0 Hz and weight 25400.0
Results/ATS26.6 - 1459460168.352141.wav.csv
ATS26.7
Search area modified due to whistle anomaly
